# Музыка больших городов

## **Оглавление**

### **[Введение](#1)**
### **1. [Обзор данных](#2)**  
### **2. [Предобработка данных](#3)**  
    2.1 Стили заголовков
    2.2 Пропуски значений
    2.3 Дубликаты
### **3. [Проверка гипотез](#7)**  
    3.1 Сравнение поведения пользователей двух столиц
    3.2 Музыка в начале и конце недели
    3.3 Жанровые предпочтения в Москва и Петербурге
### **[Итоги исследования](#11)**

## **Введение** <a id='1'></a>
На данных Яндекс.Музыки мы сравниваем поведение пользователей двух столиц.

**Цель исследования** — проверьте три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.
 
Исследование проводится в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных <a id='2'></a>

Составим первое представление о данных Яндекс.Музыки.




Импортируйте библиотеку `pandas`.

In [2]:
import pandas as pd

Прочитаем файл с данными и сохраним его в переменной `df`:

In [3]:
df = pd.read_csv('/datasets/yandex_music_project.csv')

Выводим на экран первые десять строк таблицы:

In [4]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Получаем общую информацию о таблице:

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
  userID    65079 non-null object
Track       63848 non-null object
artist      57876 non-null object
genre       63881 non-null object
  City      65079 non-null object
time        65079 non-null object
Day         65079 non-null object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Нет единого стиля написания названий.



Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных <a id='3'></a>

### Стиль заголовков
Исправим стиль заголовков.

Выведем на экран названия столбцов:

In [6]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведем названия в соответствие с единым стилем, для этого переименуем колонки:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [7]:
df.set_axis(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], axis='columns', inplace = True)

In [1]:
#или можно применить специальный метод для переименования
#df = df.rename(columns={'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day': day'}) 

Проверьте результат. Для этого ещё раз выведите на экран названия столбцов:

In [9]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений
Посчитаем, сколько в таблице пропущенных значений.

In [10]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пропущенные значения в `track` и `artist` не важны для исследования. Достаточно заменить их явными обозначениями.

Пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. Было бы правильно установить причину пропусков и восстановить данные, но такой возможности нет в учебном проекте. Поэтому заполним пропуски явными обозначениями.

Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создадим список `columns_to_replace`и цикл `for`, который переберет элементы списка и для каждого столбца выполните замену пропущенных значений:

In [11]:
columns_to_replace = ['track', 'artist', 'genre']
for value in columns_to_replace:
    df[value] = df[value].fillna('unknown')

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [12]:
df.isna().sum() # подсчёт пропусков

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты
Посчитаем явные дубликаты в таблице:

In [13]:
df.duplicated().sum()

3826

Удалим явные дубликаты:

In [14]:
df = df.drop_duplicates().reset_index(drop = True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)

Убедимся, что полностью избавились от явных дубликатов:

In [15]:
df.duplicated().sum()

0

Теперь избавьтесь от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному.

Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке.

In [16]:
df.sort_values(by='genre')['genre'].unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Просмотрим список и найдем неявные дубликаты названия `hiphop`.

Найдены следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу, напишем функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

Функция заменит каждое значение из списка `wrong_genres` на значение из `correct_genre`.

In [17]:
wrong_genres = ['hip', 'hop', 'hip-hop']
correct_genre = ['hiphop']
def replace_wrong_genres(data, wrong_genres):
    for value in wrong_genres:
         data['genre'] = data['genre'].replace(value, 'hiphop')

Вызовем функцию `replace_wrong_genres()`

In [18]:
replace_wrong_genres(df, wrong_genres)

Проверьим, что заменили неправильные названия, выведем снова отсортированный список уникальных значений столбца `genre`:

In [19]:
df['genre'].unique()

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb',
       'hiphop', 'jazz', 'postrock', 'latin', 'classical', 'metal',
       'reggae', 'tatar', 'blues', 'instrumental', 'rusrock', 'dnb',
       'türk', 'post', 'country', 'psychedelic', 'conjazz', 'indie',
       'posthardcore', 'local', 'avantgarde', 'punk', 'videogame',
       'techno', 'house', 'christmas', 'melodic', 'caucasian',
       'reggaeton', 'soundtrack', 'singer', 'ska', 'shanson', 'ambient',
       'film', 'western', 'rap', 'beats', "hard'n'heavy", 'progmetal',
       'minimal', 'contemporary', 'new', 'soul', 'holiday', 'german',
       'tropical', 'fairytail', 'spiritual', 'urban', 'gospel', 'nujazz',
       'folkmetal', 'trance', 'miscellaneous', 'anime', 'hardcore',
       'progressive', 'chanson', 'numetal', 'vocal', 'estrada', 'russian',
       'classicmetal', 'dubstep', 'club', 'deep', 'southern', 'black',
       'folkrock', 'fitness'

Сделаем дополнительную проверку с помощью функции `find_genre()`, которая принимает как параметр список с неправильными названиями.  
Цикл проходит по списку уникальных значений и сравнивает их с неправильным названием, если они совпадают, то он добавляет это название в список `result`.  
Вызываем функцию, если мы удалили все неправильные названия, список должен остаться пустой.

In [20]:
genres_list = df['genre'].unique()
result = []
def find_genre(row): 
    for value in row: 
        for element in genres_list:
            if element == value:
                result.append(value)

find_genre(wrong_genres)

[]


Еще раз смотрим информацию о таблице, чтобы убедиться, что все в порядке

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61253 entries, 0 to 61252
Data columns (total 7 columns):
user_id    61253 non-null object
track      61253 non-null object
artist     61253 non-null object
genre      61253 non-null object
city       61253 non-null object
time       61253 non-null object
day        61253 non-null object
dtypes: object(7)
memory usage: 3.3+ MB


**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Мы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения мы заменили на `'unknown'`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез <a id='7'></a>

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Посчитаем количество прослушиваний в каждом городе


In [22]:
df.groupby('city')['genre'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: genre, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитаем прослушивания в понедельник, среду и пятницу.


In [23]:
df.groupby('day')['genre'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: genre, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Теперь напишем функцию, которая объединит два эти расчёта, чтобы получить данные для каждого города в каждый из трёх дней.

Создадим функцию `number_tracks()` с одним параметром дня недели.  
Цикл сохраняет в переменную `track_list` те строки таблицы `df`, для которых значение в столбце `'day'` равно параметру `day` и одновременно значение в столбце `'city'` равно значению из списка `cities`.  
Для каждого города в переменной `track_list_count` сохраняется число значений столбца `'user_id'` и это значение добавляется в 
соответствующий список `row_moscow` или `row_spb`.  

Вызовем функцию `number_tracks()`.

In [24]:
dayweek = ['Monday', 'Wednesday', 'Friday'] #создадим список из дней недели
cities = ['Moscow', 'Saint-Petersburg'] #создадим список из городов
row_moscow = ['Moscow'] #список, в который добавим значения по москве
row_spb = ['Saint-Petersburg'] #список, в который добавим значения по санкт-петербургу

def number_tracks(day):
    for value in day: 
        track_list = df.loc[(df.loc[:, 'day'] == value) & (df.loc[:, 'city'] == cities[0])]
        track_list_count = track_list.loc[:,'user_id'].count()
        row_moscow.append(track_list_count)
        track_list = df.loc[(df.loc[:, 'day'] == value) & (df.loc[:, 'city'] == cities[1])]
        track_list_count = track_list.loc[:,'user_id'].count()
        row_spb.append(track_list_count)
    
number_tracks(dayweek)

Создадим c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые мы получили с помощью `number_tracks`.

In [31]:
data = [row_moscow, row_spb] #используем списки
columns =  ['city', 'monday', 'wednesday', 'friday']
table = pd.DataFrame(data = data, columns = columns)
print(table)

               city  monday  wednesday  friday
0            Moscow   15740      11056   15945
1  Saint-Petersburg    5614       7003    5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [32]:
moscow_general = df.loc[df.loc[:, 'city'] == 'Moscow']


In [33]:
spb_general = df.loc[df.loc[:, 'city'] == 'Saint-Petersburg']


Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, отсортированных по убыванию, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [34]:
def genre_weekday(table, day, time1, time2):
    genre_df = table.loc[(table.loc[:, 'day'] == day) & (table.loc[:, 'time'] > time1) & (table.loc[:, 'time'] < time2)]
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending = False).head(10)
    return genre_df_sorted

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [35]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [36]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [37]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            717
dance          524
rock           518
electronic     485
hiphop         238
alternative    182
world          172
classical      172
ruspop         149
rusrap         133
Name: genre, dtype: int64

In [38]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            263
rock           208
electronic     192
dance          191
hiphop         104
alternative     72
classical       71
jazz            57
rusrap          54
ruspop          53
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [39]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending = False)

Выведем на экран первые десять строк `moscow_genres`:

In [40]:
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Теперь повторим то же и для Петербурга.

Сгруппируем таблицу `spb_general` по жанру. Посчитаем прослушивания треков каждого жанра. Результат отсортируем в порядке убывания и сохраним в таблице `spb_genres`:


In [41]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending = False)

Выведем на экран первые десять строк `spb_genres`:

In [42]:
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования <a id='11'></a>

Мы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.